In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Логичтическая регрессия

Логистическая регрессия используется для задач классификации,

при этом вычисляется вероятность принадлежности

события к определенному классу.

Задача алгоритма логистической регрессии - найти

подходящие коэффициенты w при признаках x:

z = w0 + w1 * x1 + w2 * x2 +...+wm * xm

Величина z помещается в сигмоидную функцию для вычисления вероятности:

f(x) = 1 / (1 + e^-z)

Значение f(x) будет расположено на отрезке [0, 1].

f(x) - вероятность отнесения события к классу 1,

1 - f(x) - вероятность отнесения события к классу 0.

### Log loss

Алгоритм логистической регрессии минимизирует величину logloss:

logloss = -y * ln(p) - (1 - y) * ln(1 - p)

где y - истинное значение (0 или 1)

p - вычесленная алгоритмом вероятность того, что событие принадлежит классу 1

### Titanic

Для изучения классификации мы будем использовать

датасет с информацией о пассажирах **Titanic** с сайта Kaggle.com

Ссылка: https://www.kaggle.com/c/titanic/data

Нам понадобится файл под названием train.csv, который мы поместим в папку input:

In [2]:
data = pd.read_csv('./train.csv')

In [3]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Установим в качестве индекса Passengerld - так нам будет проще отслеживать, с информацией о каком пассажире мы работаем

In [4]:
data = data.set_index('PassengerId')

In [5]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
data.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Наша задача - определить, выжил ли человек при крушении Титаника.

В поле Survived выжившие пассажиры обозначены единицейб а утонувшие - нулем.

In [7]:
target = 'Survived'

In [8]:
y = data[target]

In [9]:
X = data.drop(target, axis=1)

### Изучение качества данных и их очистка

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


В поле Cabin слишком много пропущенных значений,

в этом уроке мы не будем его использовать.

Столбцы Name и Ticket также в этот раз не рассматриваем.

Удалим эти поля:

In [11]:
X = X.drop(['Cabin', 'Name', 'Ticket'], axis=1)

В поле возраст 20% значений не заполнено. Заменим пропущенные значения на значения среднего возраста.

In [12]:
mean_age = X['Age'].mean()
mean_age

29.69911764705882

In [13]:
X['Age'] = X['Age'].fillna(mean_age)

In [14]:
X['Age'].unique()

array([22.        , 38.        , 26.        , 35.        , 29.69911765,
       54.        ,  2.        , 27.        , 14.        ,  4.        ,
       58.        , 20.        , 39.        , 55.        , 31.        ,
       34.        , 15.        , 28.        ,  8.        , 19.        ,
       40.        , 66.        , 42.        , 21.        , 18.        ,
        3.        ,  7.        , 49.        , 29.        , 65.        ,
       28.5       ,  5.        , 11.        , 45.        , 17.        ,
       32.        , 16.        , 25.        ,  0.83      , 30.        ,
       33.        , 23.        , 24.        , 46.        , 59.        ,
       71.        , 37.        , 47.        , 14.5       , 70.5       ,
       32.5       , 12.        ,  9.        , 36.5       , 51.        ,
       55.5       , 40.5       , 44.        ,  1.        , 61.        ,
       56.        , 50.        , 36.        , 45.5       , 20.5       ,
       62.        , 41.        , 52.        , 63.        , 23.5 

В поле Embarked, которое является текстовым, не хватает двух значений,

заполним пропущенные значения наиболее часто встречающимся значением (мода)

In [15]:
X['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [16]:
embarked_mode = X['Embarked'].mode()[0]
embarked_mode

'S'

In [17]:
X['Embarked'] = X['Embarked'].fillna(embarked_mode)

In [18]:
X['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 55.7+ KB


### Работа с категориальными признаками

Признак пола пассажира (Sex) - категориальная переменная. Так как в ней два класса,

то мы можем представить одной колонкой со значени/ми 0 и 1:

In [20]:
X['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [21]:
X['Sex'] = (X['Sex'] == 'female').astype(int)

In [22]:
X['Sex'].value_counts()

0    577
1    314
Name: Sex, dtype: int64

Ещё одна категориальная переменная - Embarked.

Преобразуем её значения в dummy-переменные.

Теперь каждому её значению будет отведен отдельный столбец,

в котором возможны два числа:

    1 (переменная равна данному значению),
    
    0 (переменная не равна данному значению)

In [23]:
X = pd.get_dummies(X)

Переменная Embarked преобразовалась в дамми-переменные Embarked_C, Embarked_Q, Embarked_S:

In [24]:
X.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [25]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(4), uint8(3)
memory usage: 51.3 KB


In [26]:
X.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,0,22.000000,1,0,7.2500,0,0,1
2,1,1,38.000000,1,0,71.2833,1,0,0
3,3,1,26.000000,0,0,7.9250,0,0,1
4,1,1,35.000000,1,0,53.1000,0,0,1
5,3,0,35.000000,0,0,8.0500,0,0,1
6,3,0,29.699118,0,0,8.4583,0,1,0
7,1,0,54.000000,0,0,51.8625,0,0,1
8,3,0,2.000000,3,1,21.0750,0,0,1
9,3,1,27.000000,0,2,11.1333,0,0,1


Переменную Pclass можно отправить в модель в таком виде,

какая она есть сейчас, так как она представлена числами,

но так как это все-таки не количественная переменная

(например, нельзя сказать, что между классами 1 и 2 такая же "разность",

как и между классами 2 и 3), то её так же переводим в дамми-переменные.

Для этого поменяем её тип данных на category

и ещё раз применим функцию get_dummies

ко всему датафрейму.

Эта функция переведет в дамми-переменные все нечисловые признаки.

In [27]:
X['Pclass'].unique()

array([3, 1, 2])

In [28]:
X['Pclass'] = X['Pclass'].astype('category')

In [29]:
X = pd.get_dummies(X)

In [30]:
X.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3'],
      dtype='object')

In [31]:
X.head(10)

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
1,0,22.000000,1,0,7.2500,0,0,1,0,0,1
2,1,38.000000,1,0,71.2833,1,0,0,1,0,0
3,1,26.000000,0,0,7.9250,0,0,1,0,0,1
4,1,35.000000,1,0,53.1000,0,0,1,1,0,0
5,0,35.000000,0,0,8.0500,0,0,1,0,0,1
6,0,29.699118,0,0,8.4583,0,1,0,0,0,1
7,0,54.000000,0,0,51.8625,0,0,1,1,0,0
8,0,2.000000,3,1,21.0750,0,0,1,0,0,1
9,1,27.000000,0,2,11.1333,0,0,1,0,0,1


### Цель разбиения данных на тренировочный, валидационный и тестовые датасеты

В этот раз мы будем более работать  с разделением данных для модели более тщательно,

мы разобъем данные из файла train.csv на две части:

тренировочный набор данных (признаки в  X_train, целевая переменная в y_train)

и валидационный (соответственно X_valid и y_valid).

На тренировачном датасете мы будем строить модель, а на валидационном - проверять её качество.

После проверки на валидационном сете можно будет окончательно проверить модель на тестевом сете

(содержится в файле train.csv по вышеуказанной ссылке -

фактическое значение целевой переменной в тестовом наборе отсутствует,

рекомендуем самостоятельно проверить точность данных при отправке решения на странице соревнования).

In [32]:
# Параметр test_size будет определять, какую долю данных
# из train.csv мы берем для валидационного датасета
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

### Построение модели

В начале работы над любой задачей рекомендуется сначала сделать простую модель,

не затрачивая большого количества времени.

Затем можно будет оценивать новые модели, сравнивая их качество

с качеством первоначальной (базовой) модели.

Сначала посмотрим на информацию о принаках:

In [33]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 299 to 103
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sex         668 non-null    int64  
 1   Age         668 non-null    float64
 2   SibSp       668 non-null    int64  
 3   Parch       668 non-null    int64  
 4   Fare        668 non-null    float64
 5   Embarked_C  668 non-null    uint8  
 6   Embarked_Q  668 non-null    uint8  
 7   Embarked_S  668 non-null    uint8  
 8   Pclass_1    668 non-null    uint8  
 9   Pclass_2    668 non-null    uint8  
 10  Pclass_3    668 non-null    uint8  
dtypes: float64(2), int64(3), uint8(6)
memory usage: 35.2 KB


Все признаки - числовые, поэтому мы можем построить модель, используя все признаки.

In [34]:
lr = LogisticRegression()

In [35]:
lr.fit(X_train, y_train)

LogisticRegression()

In [36]:
y_pred = lr.predict(X_valid)

In [37]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0])

Метрика Accuracy (доля правильных ответов)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(y_valid, y_pred)

0.8026905829596412

Можно сравнить с Accuracy на тренировачном датасете

In [40]:
y_pred_train = lr.predict(X_train)

In [41]:
accuracy_score(y_train, y_pred_train)

0.8083832335329342

### Вычисление вероятности событий

In [42]:
y_proba = lr.predict_proba(X_valid)

In [43]:
# Вероятности событий 0 и 1 для каждого пассажира
y_proba

array([[0.88803847, 0.11196153],
       [0.7306994 , 0.2693006 ],
       [0.87015651, 0.12984349],
       [0.08991937, 0.91008063],
       [0.25062016, 0.74937984],
       [0.07826308, 0.92173692],
       [0.33446958, 0.66553042],
       [0.9052244 , 0.0947756 ],
       [0.24542475, 0.75457525],
       [0.10340605, 0.89659395],
       [0.69433675, 0.30566325],
       [0.93487211, 0.06512789],
       [0.62484706, 0.37515294],
       [0.8476136 , 0.1523864 ],
       [0.75859308, 0.24140692],
       [0.07898047, 0.92101953],
       [0.72748704, 0.27251296],
       [0.33438491, 0.66561509],
       [0.70247546, 0.29752454],
       [0.70651162, 0.29348838],
       [0.88386345, 0.11613655],
       [0.64286509, 0.35713491],
       [0.39821922, 0.60178078],
       [0.8694494 , 0.1305506 ],
       [0.89833287, 0.10166713],
       [0.92821101, 0.07178899],
       [0.56597441, 0.43402559],
       [0.72832999, 0.27167001],
       [0.9144287 , 0.0855713 ],
       [0.42638414, 0.57361586],
       [0.

Нас интересует вероятность события 1 (правый столбец) -

при получении ответа значения из него округляются до 0 или до 1

### Сохранение данных

Сохраним обработанные данные, они пригодятся для следующих уроков

In [44]:
X_train.to_pickle('X_train.pkl')
y_train.to_pickle('y_train.pkl')

X_valid.to_pickle('X_valid.pkl')
y_valid.to_pickle('y_valid.pkl')